In [1]:
%%javascript
IPython.notebook.kernel.execute('nb_name = "' + IPython.notebook.notebook_name + '"')

<IPython.core.display.Javascript object>

In [2]:
%load_ext autoreload
%autoreload
from IPython.display import clear_output

import os
os.environ['PYTHONHASHSEED'] = '0'
os.environ["NUMEXPR_NUM_THREADS"] = "1"
os.environ["MKL_NUM_THREADS"] = "1"
os.environ["OMP_NUM_THREADS"] = "1"

import sys
import argparse
import subprocess
import pdb
import time
import random
import _pickle as cPickle
import matplotlib.pyplot as plt
import glob

%matplotlib inline

import numpy as np
import pandas as pd
import tensorflow as tf

from data_structure import get_batches
from hntm import HierarchicalNeuralTopicModel
from tree import get_descendant_idxs
from evaluation import validate, get_hierarchical_affinity, get_topic_specialization, print_topic_sample
from configure import get_config

# load data & set config

In [3]:
config = get_config(nb_name)

In [4]:
os.environ["CUDA_VISIBLE_DEVICES"] = config.gpu
np.random.seed(config.seed)
random.seed(config.seed)

In [5]:
instances_train, instances_dev, instances_test, word_to_idx, idx_to_word, bow_idxs = cPickle.load(open(config.path_data,'rb'))
train_batches = get_batches(instances_train, config.batch_size)
dev_batches = get_batches(instances_dev, config.batch_size)
test_batches = get_batches(instances_test, config.batch_size)
config.dim_bow = len(bow_idxs)

In [6]:
def debug_shape(variables, model):
    sample_batch = test_batches[0][1]
    feed_dict = model.get_feed_dict(sample_batch)
    _variables = sess.run(variables, feed_dict=feed_dict)
    for _variable, variable in zip(_variables, variables):
        if hasattr(variable, 'name'):
            print(variable.name, ':', _variable.shape)
        else:
            print(_variable.shape)

def debug_value(variables, model, return_value=False):
    sample_batch = test_batches[0][1]
    feed_dict = model.get_feed_dict(sample_batch)
    _variables = sess.run(variables, feed_dict=feed_dict)

    return _variables

# run

## initialize log

In [7]:
checkpoint = []
losses_train = []
ppls_train = []
ppl_min = np.inf
epoch = 0
train_batches = get_batches(instances_train, config.batch_size, iterator=True)

log_df = pd.DataFrame(columns=pd.MultiIndex.from_tuples(
                    list(zip(*[['','','','TRAIN:','','','','','VALID:','','','','','TEST:','', 'SPEC:', '', '', 'HIER:', ''],
                            ['Time','Ep','Ct','LOSS','PPL','NLL','KL','REG','LOSS','PPL','NLL','KL','REG','LOSS','PPL', '1', '2', '3', 'CHILD', 'OTHER']]))))

cmd_rm = 'rm -r %s' % config.dir_model
res = subprocess.call(cmd_rm.split())
cmd_mk = 'mkdir %s' % config.dir_model
res = subprocess.call(cmd_mk.split())

def update_checkpoint(config, checkpoint, global_step):
    checkpoint.append(config.path_model + '-%i' % global_step)
    if len(checkpoint) > config.max_to_keep:
        path_model = checkpoint.pop(0) + '.*'
        for p in glob.glob(path_model):
            os.remove(p)
    cPickle.dump(checkpoint, open(config.path_checkpoint, 'wb'))

## initialize model

In [8]:
if 'sess' in globals(): sess.close()
model = HierarchicalNeuralTopicModel(config)
sess = tf.Session(config=tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1))
sess.run(tf.global_variables_initializer())
saver = tf.train.Saver(max_to_keep=config.max_to_keep)
update_tree_flg = False

## train & validate model

In [ ]:
time_start = time.time()
while epoch < config.n_epochs:
    # train
    for ct, batch in train_batches:
        feed_dict = model.get_feed_dict(batch)
        _, loss_batch, topic_loss_recon_batch, topic_loss_kl_batch, topic_loss_reg_batch, ppls_batch, global_step_log = \
        sess.run([model.opt, model.loss, model.topic_loss_recon, model.topic_loss_kl, model.topic_loss_reg, model.topic_ppls, tf.train.get_global_step()], feed_dict = feed_dict)

        losses_train += [[loss_batch, topic_loss_recon_batch, topic_loss_kl_batch, topic_loss_reg_batch]]
        ppls_train += list(ppls_batch)

        if global_step_log % config.log_period == 0:
            # validate
            loss_train, topic_loss_recon_train, topic_loss_kl_train, topic_loss_reg_train = np.mean(losses_train, 0)
            ppl_train = np.exp(np.mean(ppls_train))
            loss_dev, topic_loss_recon_dev, topic_loss_kl_dev, topic_loss_reg_dev, ppl_dev, probs_topic_dev = validate(sess, dev_batches, model)

            # test
            if ppl_dev < ppl_min:
                ppl_min = ppl_dev
                loss_test, _, _, _, ppl_test, _ = validate(sess, test_batches, model)
                saver.save(sess, config.path_model, global_step=global_step_log)
                cPickle.dump(config, open(config.path_config % global_step_log, 'wb'))
                update_checkpoint(config, checkpoint, global_step_log)
            
            # visualize topic
            topics_freq_indices = np.argsort(sess.run(model.topic_bow), 1)[:, ::-1][:, :config.n_freq]
            topics_freq_idxs = bow_idxs[topics_freq_indices]
            topic_freq_tokens = {topic_idx: [idx_to_word[idx] for idx in topic_freq_idxs] for topic_idx, topic_freq_idxs in zip(model.topic_idxs, topics_freq_idxs)}
            topic_prob_topic = {topic_idx: prob_topic for topic_idx, prob_topic in zip(model.topic_idxs, probs_topic_dev)}
            descendant_idxs = {parent_idx: get_descendant_idxs(model, parent_idx) for parent_idx in model.topic_idxs}
            recur_prob_topic = {parent_idx: np.sum([topic_prob_topic[child_idx] for child_idx in recur_child_idxs]) for parent_idx, recur_child_idxs in descendant_idxs.items()}
            
            depth_specs = get_topic_specialization(sess, model, instances_test)
            hierarchical_affinities = get_hierarchical_affinity(sess, model)
            
            # log
            clear_output()
            time_log = int(time.time() - time_start)
            log_series = pd.Series([time_log, epoch, ct, \
                    '%.2f'%loss_train, '%.0f'%ppl_train, '%.2f'%topic_loss_recon_train, '%.2f'%topic_loss_kl_train, '%.2f'%topic_loss_reg_train, \
                    '%.2f'%loss_dev, '%.0f'%ppl_dev, '%.2f'%topic_loss_recon_dev, '%.2f'%topic_loss_kl_dev, '%.2f'%topic_loss_reg_dev, \
                    '%.2f'%loss_test, '%.0f'%ppl_test, \
                    '%.2f'%depth_specs[1], '%.2f'%depth_specs[2], '%.2f'%depth_specs[3], \
                    '%.2f'%hierarchical_affinities[0], '%.2f'%hierarchical_affinities[1]],
                    index=log_df.columns)
            log_df.loc[global_step_log] = log_series
            display(log_df)
            cPickle.dump(log_df, open(os.path.join(config.path_log), 'wb'))
            print_topic_sample(sess, model, topic_prob_topic=topic_prob_topic, recur_prob_topic=recur_prob_topic, topic_freq_tokens=topic_freq_tokens)

            # update tree
            if not config.static:
                config.tree_idxs, update_tree_flg = model.update_tree(topic_prob_topic, recur_prob_topic)
                if update_tree_flg:
                    print(config.tree_idxs)
                    name_variables = {tensor.name: variable for tensor, variable in zip(tf.global_variables(), sess.run(tf.global_variables()))} # store paremeters
                    if 'sess' in globals(): sess.close()
                    model = HierarchicalNeuralTopicModel(config)
                    sess = tf.Session(config=tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1))
                    name_tensors = {tensor.name: tensor for tensor in tf.global_variables()}
                    sess.run([name_tensors[name].assign(variable) for name, variable in name_variables.items()]) # restore parameters
                    saver = tf.train.Saver(max_to_keep=1)
                
            time_start = time.time()

    train_batches = get_batches(instances_train, config.batch_size, iterator=True)
    epoch += 1

loss_dev, topic_loss_recon_dev, topic_loss_kl_dev, topic_loss_reg_dev, ppl_dev, probs_topic_dev = validate(sess, dev_batches, model)
topics_freq_indices = np.argsort(sess.run(model.topic_bow), 1)[:, ::-1][:, :config.n_freq]
topics_freq_idxs = bow_idxs[topics_freq_indices]
topic_freq_tokens = {topic_idx: [idx_to_word[idx] for idx in topic_freq_idxs] for topic_idx, topic_freq_idxs in zip(model.topic_idxs, topics_freq_idxs)}
topic_prob_topic = {topic_idx: prob_topic for topic_idx, prob_topic in zip(model.topic_idxs, probs_topic_dev)}
descendant_idxs = {parent_idx: get_descendant_idxs(model, parent_idx) for parent_idx in model.topic_idxs}
recur_prob_topic = {parent_idx: np.sum([topic_prob_topic[child_idx] for child_idx in recur_child_idxs]) for parent_idx, recur_child_idxs in descendant_idxs.items()}
display(log_df)
print_topic_sample(sess, model, topic_prob_topic=topic_prob_topic, recur_prob_topic=recur_prob_topic, topic_freq_tokens=topic_freq_tokens)

TRAIN:                               VALID:        \
       Time   Ep   Ct     LOSS   PPL      NLL     KL   REG     LOSS   PPL   
5000    100    8  463  9175.77  1876  9157.67  17.82  0.27  9156.08  1791   
10000    92   17  360  9131.72  1809  9112.23  19.30  0.22  9136.03  1764   
15000    91   26  257  9111.77  1780  9091.72  19.87  0.19  9131.22  1756   
20000    91   35  154  9099.94  1763  9079.66  20.15  0.18  9127.53  1751   
25000    89   44   51  9091.84  1752  9071.42  20.31  0.17  9124.96  1748   
30000    89   52  515  9086.05  1744  9065.49  20.40  0.16  9124.32  1747   
35000    88   61  412  9081.68  1738  9061.18  20.46  0.15  9123.49  1745   
40000    88   70  309  9078.19  1732  9057.69  20.51  0.15  9123.15  1745   
45000    85   79  206  9075.20  1728  9054.66  20.54  0.14  9121.44  1743   
50000    84   88  103  9072.60  1725  9052.08  20.57  0.14  9120.72  1743   
55000    75   97    0  9070.25  1722  9049.79  20.59  0.14  9121.31  1744   
60000    75  105  464  9068.35  1719  9047.88  20.61  0.13  9121.82  1743   
65000    77  114  361  9066.80  1716  9046.26  20.63  0.13  9121.00  1743   
70000    80  123  258  9065.33  1714  9044.70  20.64  0.13  9121.51  1744   
75000    76  132  155  9063.95  1712  9043.28  20.65  0.13  9118.39  1740   
80000    76  141   52  9062.66  1711  9041.94  20.66  0.13  9120.56  1742   
85000    77  149  516  9061.53  1709  9040.78  20.66  0.12  9119.63  1740   
90000    83  158  413  9060.53  1707  9039.76  20.67  0.12  9120.26  1742   
95000    79  167  310  9059.63  1706  9038.84  20.67  0.12  9121.33  1743   
100000   78  176  207  9058.73  1705  9037.93  20.67  0.12  9120.45  1742   
105000   76  185  104  9057.89  1704  9037.09  20.67  0.12  9120.83  1743   
110000   77  194    1  9057.06  1703  9036.25  20.67  0.12  9121.86  1744   
115000   77  202  465  9056.35  1702  9035.53  20.67  0.12  9122.33  1745   
120000   77  211  362  9055.74  1701  9034.89  20.67  0.12  9121.30  1744   
125000   77  220  259  9055.14  1700  9034.20  20.67  0.12  9120.69  1743   
130000   87  229  156  9054.56  1699  9033.53  20.67  0.12  9120.14  1742   
135000   75  238   53  9053.98  1698  9032.86  20.67  0.11  9120.11  1742   
140000   77  246  517  9053.46  1697  9032.27  20.66  0.11  9120.68  1743   
145000   79  255  414  9053.00  1697  9031.74  20.66  0.11  9120.62  1742   
150000   86  264  311  9052.58  1696  9031.25  20.65  0.11  9122.20  1744   
155000   82  273  208  9052.14  1695  9030.76  20.65  0.11  9121.59  1745   
160000   86  282  105  9051.72  1695  9030.29  20.64  0.11  9121.12  1743   
165000   78  291    2  9051.28  1694  9029.79  20.64  0.11  9121.95  1745   
170000   78  299  466  9050.89  1693  9029.36  20.64  0.11  9123.14  1746   
175000   78  308  363  9050.56  1693  9028.99  20.63  0.11  9121.28  1744   
180000   78  317  260  9050.23  1692  9028.61  20.63  0.11  9122.27  1745   
185000   77  326  157  9049.88  1692  9028.23  20.62  0.11  9120.53  1743   
190000   81  335   54  9049.53  1691  9027.84  20.62  0.11  9122.55  1745   
195000   81  343  518  9049.21  1691  9027.49  20.62  0.11  9122.39  1745   
200000   80  352  415  9048.92  1690  9027.18  20.62  0.11  9122.32  1744   
205000   81  361  312  9048.66  1690  9026.90  20.61  0.11  9122.04  1745   
210000   79  370  209  9048.39  1690  9026.60  20.61  0.11  9122.55  1746   
215000   79  379  106  9048.12  1689  9026.31  20.61  0.11  9123.29  1747   
220000   87  388    3  9047.83  1689  9026.00  20.61  0.11  9122.92  1746   
225000   96  396  467  9047.58  1688  9025.74  20.60  0.11  9123.05  1745   

                                TEST:       SPEC:             HIER:        
            NLL     KL   REG     LOSS   PPL     1     2     3 CHILD OTHER  
5000    9136.12  19.77  0.19  9156.00  1791  0.39  0.45  0.41  0.30  0.24  
10000   9115.48  20.40  0.15  9135.53  1763  0.41  0.51  0.46  0.24  0.20  
15000   9110.29  20.80  0.13  9130.96  1756  0.41  0.52  0.48  0.23  0.19  
20000   9106.58  20.83  

0 R: 1.000 P: 0.111 speech user dialogue human utterance utterances speaker systems recognition users
   1 R: 0.234 P: 0.079 semantic verb structure syntactic lexical type subject noun representation meaning
     11 R: 0.078 P: 0.078 tree parsing dependency parser grammar node rules trees parse rule
     12 R: 0.077 P: 0.077 languages morphological errors english pos character error chinese tags tag
   2 R: 0.191 P: 0.057 resources domain linguistic annotation ontology knowledge languages project concepts semantic
     21 R: 0.062 P: 0.062 translation english source alignment target phrase pairs parallel translations sentences
     22 R: 0.072 P: 0.072 sense semantic wordnet lexical senses noun verb context nouns terms
   3 R: 0.177 P: 0.033 entity entities named mentions coreference mention extraction names features resolution
     31 R: 0.091 P: 0.091 models probability algorithm training method distribution function parameters probabilities sequence
     32 R: 0.053 P: 0.053 documen

# analysis

In [34]:
def get_topic_year():
    probs_topics = []
    years = []
    for i, train_batch in train_batches:
        probs_topics_batch = sess.run(model.prob_topic, feed_dict=model.get_feed_dict(train_batch, mode='test'))
        years_batch = [instance.year for instance in train_batch]
        probs_topics += [probs_topics_batch]
        years += years_batch
    probs_topics = np.concatenate(probs_topics)
    years = np.array(years)

    topic_years = years.dot(probs_topics) / np.sum(probs_topics, 0)
    topic_year = {model.topic_idxs[i]: year for i, year in enumerate(topic_years)}
    return topic_year

In [55]:
probs_topics.shape, years.shape

((36288, 16), (36288,))

In [59]:
topic_year

{0: 2006.1405287293783,
 1: 2015.073065684627,
 2: 2006.6358226095088,
 3: 2010.2605308626469,
 4: 2011.1153424999409,
 5: 2010.2201559271937,
 11: 2010.638231762466,
 12: 2013.6412160983643,
 21: 2009.3432210176409,
 22: 2003.0377479786164,
 31: 2010.7845425693845,
 32: 2007.4959372377405,
 41: 2008.9745451250799,
 42: 2009.024079592822,
 51: 2007.963976828227,
 52: 2008.7701026307714}

In [64]:
def print_topic_sample(sess, model, topic_prob_topic=None, recur_prob_topic=None, topic_freq_tokens=None, topic_year=None, parent_idx=0, depth=0):
    if depth == 0: # print root
        assert len(topic_prob_topic) == len(recur_prob_topic) == len(topic_freq_tokens) == len(topic_year)
        freq_tokens = topic_freq_tokens[parent_idx]
        recur_topic = recur_prob_topic[parent_idx]
        prob_topic = topic_prob_topic[parent_idx]
        year = topic_year[parent_idx]
        print(parent_idx, 'Avg Year: %i' % year, ' '.join(freq_tokens))
    
    child_idxs = model.tree_idxs[parent_idx]
    depth += 1
    for child_idx in child_idxs:
        freq_tokens = topic_freq_tokens[child_idx]
        recur_topic = recur_prob_topic[child_idx]
        prob_topic = topic_prob_topic[child_idx]
        year = topic_year[child_idx]
        print('  '*depth, child_idx, 'Avg Year: %i' % year, ' '.join(freq_tokens))
        
        if child_idx in model.tree_idxs: 
            print_topic_sample(sess, model, topic_prob_topic=topic_prob_topic, recur_prob_topic=recur_prob_topic, topic_freq_tokens=topic_freq_tokens, topic_year=topic_year, parent_idx=child_idx, depth=depth)

In [65]:
print_topic_sample(sess, model, topic_prob_topic=topic_prob_topic, recur_prob_topic=recur_prob_topic, topic_freq_tokens=topic_freq_tokens, topic_year=topic_year)

0 Avg Year: 2006 user dialogue knowledge utterance generation domain action state systems human
   1 Avg Year: 2015 models embeddings neural vector network training vectors representations embedding representation
     11 Avg Year: 2010 document documents question topic query sentences questions answer method similarity
     12 Avg Year: 2013 sentiment features tweets negative positive classification analysis social opinion polarity
   2 Avg Year: 2006 tree parsing dependency parser grammar node trees parse rules nodes
     21 Avg Year: 2009 event relations annotation relation discourse events argument semantic annotated temporal
     22 Avg Year: 2003 verb semantic syntactic structure noun lexical verbs rules grammar phrase
   3 Avg Year: 2010 training models algorithm probability features feature function learning performance distribution
     31 Avg Year: 2010 features feature training performance entity learning classification classifier entities class
     32 Avg Year: 2007 errors